In [ ]:
import os
import numpy as np
import open3d as o3d
from monai.data import image_reader
from skimage.segmentation import find_boundaries

In [ ]:
myReader = image_reader.ITKReader()
# Or test with ellipsoid file = "../data/ellipsoid_data/image_surf/surf_1.nrrd"
img = myReader.read("../data/hippocampi/groupA_01_hippo.gipl.gz") 

In [ ]:
data, meta = myReader.get_data(img)
data = np.asarray(data, dtype=np.uint8)

In [ ]:
type(data)

In [ ]:
data.shape

In [ ]:
type(data[0][0][0])

In [ ]:
# We only need the boundary points as input to the point2skel model
bounds = find_boundaries(data)

In [ ]:
print(bounds.shape)

In [ ]:
# Need to find a faster way to do this!
def get_point_set(image):
    pointSet = np.argwhere(image > 0) # since binary image
    pointSet = np.array(pointSet, dtype=np.float32)
    return pointSet

In [ ]:
pts = get_point_set(bounds)

In [ ]:
print(pts.shape)
print(np.max(pts, axis=0))
print(np.min(pts, axis=0))

In [ ]:
# pts /= data.shape
pts /= bounds.shape
print(np.max(pts, axis=0))
print(np.min(pts, axis=0))

In [ ]:
num_pix = data.shape[0] * data.shape[1] * data.shape[2]
print("Load factor:", pts.shape[0] / num_pix) # Image size is (250, 250, 250)

# Get random samples
target_count = 2000
idxs = np.random.randint(pts.shape[0], size=target_count)
pts = pts[idxs, :]

In [ ]:
def normalize(pts):
    pts -= np.mean(pts, axis=0)
    max_dist = np.max(np.linalg.norm(pts, axis=1))
    pts /= max_dist
    return pts

In [ ]:
pts = normalize(pts)

In [ ]:
print(np.max(pts, axis=0))
print(np.min(pts, axis=0))
print(np.mean(pts, axis=0))

In [ ]:

def save_ply_points(points, path):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    o3d.io.write_point_cloud(path, pcd)


In [ ]:
save_ply_points(pts, "test_bound.ply")